---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    result = ['PA','SW_L','SW_L','PA','SW_H']
    return result

#graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
import datetime
def loginfo(msg):
    print('[' + str(datetime.datetime.now()) + '] - ' + str(msg))
        
def salary_predictions():
    # Initialize the dataframe, using the nodes as the index
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    # add features
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    #nx.cn_soundarajan_hopcroft(G)
    #print(df.head())
    X = df[df['ManagementSalary'].notnull()]
    y = X['ManagementSalary']
    X = X[['clustering', 'degree']]
    Z = df[df['ManagementSalary'].isnull()]
    Z = Z[['clustering','degree']]

    loginfo('Scaling features...')
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_tmp = scaler.fit_transform(X)
    Z_tmp = scaler.transform(Z)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X_tmp, y, random_state=0)

    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier(n_jobs=-1, 
                                 max_features='auto', 
                                 n_estimators=5, 
                                 max_depth=2)

    clf = LogisticRegression(C=0.1)

    clf.fit(X_train, y_train)
    loginfo('--> Accuracy of classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
    loginfo('--> Accuracy of classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))

    from sklearn.metrics import precision_recall_curve, roc_auc_score, roc_curve
    from sklearn.model_selection import cross_val_predict 

    y_train_probas = clf.predict_proba(X_train)
    y_test_probas = clf.predict_proba(X_test)
    z_test_probas = clf.predict_proba(Z_tmp)

    # scores = proba of positive class
    y_train_scores = y_train_probas[:, 1]
    y_test_scores = y_test_probas[:, 1]
    z_test_scores = z_test_probas[:, 1]

    roc_auc_train = roc_auc_score(y_train, y_train_scores)
    roc_auc_test = roc_auc_score(y_test, y_test_scores)
    loginfo('--> ROC AUC tmp train score: {:.2f}'.format(roc_auc_train))
    loginfo('--> ROC AUC tmp test score: {:.2f}'.format(roc_auc_test))

    Z['proba'] = z_test_scores
    result = Z['proba']
    
    return result

#salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [57]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


# add features
df = future_connections
df['CommonNeighbors'] = df.index.map(lambda conn: len(list(nx.common_neighbors(G, conn[0], conn[1]))))
df['JaccardCoefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
df['RessourceAllocation'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
df['AdamicAdar'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
df['PreferentialAttachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
df['SoundarajanHopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, df.index, community='Department')]
df['RaSoundarajanHopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, df.index, community='Department')]
df.head()

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [58]:
def new_connections_predictions():
    df = future_connections
    df['CommonNeighbors'] = df.index.map(lambda conn: len(list(nx.common_neighbors(G, conn[0], conn[1]))))
    df['JaccardCoefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['RessourceAllocation'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    df['AdamicAdar'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    df['PreferentialAttachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['SoundarajanHopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, df.index, community='Department')]
    df['RaSoundarajanHopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, df.index, community='Department')]

    # prepare inputs
    X = df[df['Future Connection'].notnull()]
    y = X['Future Connection']
    cols = []
    cols.append('CommonNeighbors')
    cols.append('JaccardCoefficient') 
    cols.append('RessourceAllocation') 
    cols.append('AdamicAdar') 
    cols.append('PreferentialAttachment') 
    cols.append('SoundarajanHopcroft') 
    cols.append('RaSoundarajanHopcroft') 
    X = X[cols]
    Z = df[df['Future Connection'].isnull()]
    Z = Z[cols]
    loginfo('Training set: ' + str(len(X)))
    loginfo('Target set: ' + str(len(Z)))

    loginfo('Scaling features...')
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_tmp = scaler.fit_transform(X)
    Z_tmp = scaler.transform(Z)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X_tmp, y, random_state=0)

    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier(n_jobs=-1, 
                                 max_features='auto', 
                                 n_estimators=5, 
                                 max_depth=3)

    clf.fit(X_train, y_train)
    loginfo('--> Accuracy of classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
    loginfo('--> Accuracy of classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))

    from sklearn.metrics import precision_recall_curve, roc_auc_score, roc_curve
    from sklearn.model_selection import cross_val_predict 

    y_train_probas = clf.predict_proba(X_train)
    y_test_probas = clf.predict_proba(X_test)
    z_test_probas = clf.predict_proba(Z_tmp)

    # scores = proba of positive class
    y_train_scores = y_train_probas[:, 1]
    y_test_scores = y_test_probas[:, 1]
    z_test_scores = z_test_probas[:, 1]

    roc_auc_train = roc_auc_score(y_train, y_train_scores)
    roc_auc_test = roc_auc_score(y_test, y_test_scores)
    loginfo('--> ROC AUC tmp train score: {:.2f}'.format(roc_auc_train))
    loginfo('--> ROC AUC tmp test score: {:.2f}'.format(roc_auc_test))

    Z['proba'] = z_test_scores
    result = Z['proba']
    
    return result

In [59]:
#new_connections_predictions()

[2017-12-26 15:47:29.665832] - Training set: 366334
[2017-12-26 15:47:29.666510] - Target set: 122112
[2017-12-26 15:47:29.666910] - Scaling features...
[2017-12-26 15:47:30.954234] - --> Accuracy of classifier on training set: 0.96
[2017-12-26 15:47:31.107770] - --> Accuracy of classifier on test set: 0.96
[2017-12-26 15:47:31.499366] - --> ROC AUC tmp train score: 0.91
[2017-12-26 15:47:31.499892] - --> ROC AUC tmp test score: 0.91


(107, 348)    0.031673
(542, 751)    0.017316
(20, 426)     0.582893
(50, 989)     0.017316
(942, 986)    0.017316
(324, 857)    0.017316
(13, 710)     0.192909
(19, 271)     0.192909
(319, 878)    0.017316
(659, 707)    0.017316
(49, 843)     0.017316
(208, 893)    0.017316
(377, 469)    0.017316
(405, 999)    0.017316
(129, 740)    0.017316
(292, 618)    0.031673
(239, 689)    0.017316
(359, 373)    0.017316
(53, 523)     0.031673
(276, 984)    0.017316
(202, 997)    0.017316
(604, 619)    0.067257
(270, 911)    0.017316
(261, 481)    0.031673
(200, 450)    0.835024
(213, 634)    0.017316
(644, 735)    0.067257
(346, 553)    0.017316
(521, 738)    0.017316
(422, 953)    0.017316
                ...   
(672, 848)    0.017316
(28, 127)     0.985852
(202, 661)    0.017316
(54, 195)     0.985852
(295, 864)    0.017316
(814, 936)    0.017316
(839, 874)    0.017316
(139, 843)    0.017316
(461, 544)    0.017316
(68, 487)     0.017316
(622, 932)    0.017316
(504, 936)    0.017316
(479, 528) 